In [ ]:
## download assets
# bash benchmark/download_wiki20m.sh
# bash benchmark/download_wiki80.sh
# bash benchmark/download_nyt10m.sh
# bash benchmark/download_fewrel.sh
# bash benchmark/download_nyt10.sh
# bash benchmark/download_semeval.sh
# bash pretrain/download_bert.sh
# bash pretrain/download_glove.sh

# Easy Start

In [3]:
import opennre
model = opennre.get_model('wiki80_cnn_softmax')
# manually download the pretrain model
# https://thunlp.oss-cn-qingdao.aliyuncs.com/opennre/pretrain/nre/wiki80_cnn_softmax.pth.tar
model.infer({'text': 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Aed Uaridnach (died 612).', 'h': {'pos': (18, 46)}, 't': {'pos': (78, 91)}})

2022-04-18 10:52:10,442 - root - INFO - Initializing word embedding with word2vec.


('father', 0.5302285552024841)

# Chinese_wwm

## Load the pre-train LM
1. download `Chinese_wmm` from [GitHub](https://github.com/ymcui/Chinese-BERT-wwm)
2. extract the files and paste them to `opennre/pretrain/chinese_wwm`
3. rename `bert_config.json` to `config.json`

## Add new datasets in benchmark
1. download dataset from [GitHub](https://github.com/taorui-plus/OpenNRE)
2. add files into `./opennre/benchmark`

## Train Chinese_wwm

1. append `people-relation` to `train_supervised_..._.py` line 35
2. line 66: `root_path = os.path.join(os.getcwd(), 'opennre')`
3. comment `opennre.download(args.dataset, root_path=root_path)` line 77 in `train_supervised_..._.py`
4. add `download_people_realtion` function in `pretrain.py`
5. enter the command in Terminal
    ```
    python example/train_supervised_chinese_wwm.py \
        --pretrain_path {os.getcwd()}/opennre/pretrain/chinese_wwm \
        --dataset people-relation
    ```

In [ ]:
python example/train_supervised_chinese_wwm.py \
    --pretrain_path {os.getcwd()}/opennre/pretrain/chinese_wwm \
    --dataset people-relation

# Model Inference

In [ ]:

import opennre
model = opennre.get_model('chinese_wwm_entity')
# model.cuda() # enable GPU

# Load data
import os, json

cwd = os.getcwd()
file_path = os.path.join(cwd, 'data.txt')

with open(file_path, 'r', encoding = 'utf-8') as f:
    new_data = f.readlines()

# preprocess
new_data = [i.strip().replace("'", '"') for i in new_data]
new_data = [i.replace("(", '[') for i in new_data]
new_data = [i.replace(")", ']') for i in new_data]
new_data = [json.loads(i) for i in new_data]

def batch_infer(l: list):
    """
    infer the list
    """

    for i in l:
        sen = i['text']
        head = i['h']['pos']
        head = sen[head[0] : head[1]]
        tail = i['t']['pos']
        tail = sen[tail[0] : tail[1]]
        pred = model.infer(i)

        print(f'The head is {head}\nThe tail is {tail}\nThe prediction is {pred}\nThe sentence is\n\t {sen}\n\n')
    
    return 

batch_infer(new_data)